# Simulating Random Walk Processes in Networks

In [1]:
import pathpy as pp
from pprint import pprint

We first generate a directed network with weighted edges:

In [2]:
n = pp.Network(directed=False)
n.add_edge('a', 'b', weight=1, uid='a-b')
n.add_edge('b', 'c', weight=1, uid='b-c')
n.add_edge('c', 'a', weight=2, uid='c-a')
n.add_edge('c', 'd', weight=1, uid='c-d')
n.add_edge('d', 'a', weight=1, uid='d-a')

To initialize a random walk process, we can generate a `RandomWalk` instance on the network. Specifying the `weight` attribute will lead to a biased random walk, where the transition probabilities are weighted by the respective numerical property:

In [3]:
rw = pp.processes.RandomWalk(n, weight='weight')

We can inspect the transition matrix of the random walk process as follows. Using the function `matrix_pd` will return the matrix as a (nicely formatted) pandas DataFrame, while `matrix` returns the internal sparse matrix representation.

In [4]:
print(rw.matrix_pd())

     a     b    c     d
a  0.0  0.25  0.5  0.25
b  0.5  0.00  0.5  0.00
c  0.5  0.25  0.0  0.25
d  0.5  0.00  0.5  0.00


In [5]:
pprint(rw.matrix)

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>


We can use the method `generate_walk` of the random walk instance to generate a random walk with a given length. If we specify `start_node`, the walk will start from the given node object. The method returns a `Path` object.

In [6]:
p = rw.generate_walk(steps=10, start_node=n.nodes['a'])
print(p)
pprint([v.uid for v in p.nodes ]) 

Uid:			0x7fb33813a1d0
Type:			Path
Number of unique nodes:	3
Number of unique edges:	4
Path length (# edges):	10
['a', 'a', 'd', 'a', 'b', 'b', 'a', 'a', 'a', 'a', 'a']


If we omit the `start_node` argument, a random node will be chosen as start node.

In [7]:
p = rw.generate_walk(steps=10)
print(p)
pprint([v.uid for v in p.nodes ]) 

Uid:			0x7fb358bc7690
Type:			Path
Number of unique nodes:	2
Number of unique edges:	3
Path length (# edges):	10
['a', 'a', 'c', 'c', 'c', 'c', 'a', 'a', 'a', 'a', 'a']


We can generate a `PathCollection` that contains a given number of random walks with a given length specified by `steps_per_walk`. If the `start_nodes` argument is a numeric, the given number of walks will be generated, each walk starting from a random node.

In [8]:
pc = rw.generate_walks(steps_per_walk=10, start_nodes=2)
for p in pc:
    print([ v.uid for v in p.nodes ])

['d', 'd', 'd', 'd', 'c', 'c', 'd', 'd', 'd', 'd', 'c']
['b', 'c', 'c', 'c', 'a', 'a', 'a', 'a', 'd', 'd', 'a']


If `start_nodes` is a list of `Node` objects, a single random walk will be generated for each start node. To generate a single random walk starting in each node of the network, we can write:

In [9]:
pc = rw.generate_walks(steps_per_walk=10, start_nodes=n.nodes)
for p in pc:
    print([ v.uid for v in p.nodes ])

['a', 'a', 'd', 'a', 'a', 'a', 'a', 'd', 'a', 'b', 'c']
['b', 'b', 'b', 'b', 'a', 'a', 'a', 'a', 'b', 'c', 'd']
['c', 'c', 'c', 'a', 'b', 'b', 'a', 'a', 'b', 'b', 'a']
['d', 'd', 'd', 'd', 'a', 'b', 'c', 'd', 'd', 'a', 'a']


We can also use an iterator interface to iteratively perform random walk steps. I neach step, we can assess the current state of the random walk at time t.

In [10]:
for v in rw.walk(10, start_node=n.nodes['a']):
    print('Current node = {0}'.format(v))
    print('Current time = {0}'.format(rw.t))
    print(rw.visitation_frequencies())

Current node = b
Current time = 1
[0.5 0.5 0.  0. ]
Current node = c
Current time = 2
[0.33333333 0.33333333 0.33333333 0.        ]
Current node = d
Current time = 3
[0.25 0.25 0.25 0.25]
Current node = a
Current time = 4
[0.4 0.2 0.2 0.2]
Current node = c
Current time = 5
[0.33333333 0.16666667 0.33333333 0.16666667]
Current node = a
Current time = 6
[0.42857143 0.14285714 0.28571429 0.14285714]
Current node = c
Current time = 7
[0.375 0.125 0.375 0.125]
Current node = a
Current time = 8
[0.44444444 0.11111111 0.33333333 0.11111111]
Current node = c
Current time = 9
[0.4 0.1 0.4 0.1]
Current node = a
Current time = 10
[0.45454545 0.09090909 0.36363636 0.09090909]


The visitation probabilities after t steps for a given start node can be computed using the `visitation_probabilities` function.

In [11]:
rw.visitation_probabilities(0, start_node=n.nodes['a'])

matrix([[1., 0., 0., 0.]])

In [12]:
rw.visitation_probabilities(1, start_node=n.nodes['a'])

matrix([[0.  , 0.25, 0.5 , 0.25]])

In [13]:
rw.visitation_probabilities(100, start_node=n.nodes['a'])

matrix([[0.33333333, 0.16666667, 0.33333333, 0.16666667]])

The stationary visitation probabilities for $t \rightarrow \infty$ can be computed as follows:

In [14]:
rw.stationary_probabilities()

array([0.33333333, 0.16666667, 0.33333333, 0.16666667])